- db 조작을 위한 설정

In [5]:
!pip install pymysql > /dev/null
from google.colab import files
uploaded = files.upload()
filename = list(uploaded.keys())[0]

Saving mysql.json to mysql.json


In [6]:
import json
with open(filename) as fp:
    config_str = fp.read()
config = json.loads(config_str)

In [7]:
import pymysql
conn = pymysql.connect(
    host=config['host'], 
    user=config['user'], 
    password=config['password'], 
    database=config['database'], 
    port=config['port']
)
#conn = pymysql.connect(**config)

In [8]:
import random
random.seed(2021)

In [10]:
sql_select = """
select uid, uname, email,
    date_format(reg_date, '%Y-%m-%d %h:%i') As reg_date
    from users where is_deleted = 0 order by reg_date;
"""

In [11]:
cur = conn.cursor()
cur.execute(sql_select)
rows = cur.fetchmany(3)
rows

(('admin', '관리자', None, '2021-08-25 01:28'),
 ('eskim', '김은숙', None, '2021-08-25 01:41'),
 ('wjlee', '이우정', None, '2021-08-25 01:41'))

In [14]:
sql_ct_pd = '''
 CREATE TABLE if NOT exists products(
        pid INT PRIMARY KEY AUTO_INCREMENT,
        pname VARCHAR(40) NOT NULL,
        pprice INT UNSIGNED NOT NULL,
        pcategory VARCHAR(20) NOT NULL,
        pcost INT UNSIGNED NOT NULL
    ) AUTO_INCREMENT=11;
'''

In [15]:
cur = conn.cursor()
cur.execute(sql_ct_pd)

0

In [16]:
camping_goods = ['텐트','타프','천막','침낭','매트','랜턴','취사용품',
                 '아이스박스','보조배터리','워터저그']
car_goods = ['블랙박스','내비게이션','하이패스','후방카메라','자동차TV',
             '카오디오','핸즈프리','헤드유닛','방진매트','윤활방청제']
health_goods = ['러닝머신','웨이트기구','복근운동기구','벨트','거꾸리',
                '훌라후프','트램펄린','스텝퍼','로잉머신','헬스사이클']
category_names = ['캠핑용품','자동차용품','헬스용품']

In [19]:
for item in camping_goods:
    price = random.randint(100,999) * 100
    rate = random.randint(85, 98)
    cost = int(round(price * rate / 1000) * 10)
    print(item, price, '캠핑', cost)

텐트 95600 캠핑 87000
타프 97800 캠핑 92910
천막 65700 캠핑 58470
침낭 35300 캠핑 33540
매트 13500 캠핑 12420
랜턴 58500 캠핑 54990
취사용품 16500 캠핑 14850
아이스박스 37600 캠핑 33460
보조배터리 58600 캠핑 50400
워터저그 26500 캠핑 23060


In [20]:
sql = "insert into products(pname,pprice,pcategory,pcost) values(%s,%s,%s,%s);"
for i, item_list in enumerate([camping_goods, car_goods, health_goods]):
    for item in item_list:
        price = random.randint(100,999) * 100
        rate = random.randint(85, 98)
        cost = int(round(price * rate / 1000) * 10)
        cur.execute(sql, (item,price,category_names[i],cost))
conn.commit()

In [21]:
sql_ct_sale = '''
 CREATE TABLE if NOT EXISTS sales(
        sid INT PRIMARY KEY AUTO_INCREMENT,
        sdate DATE NOT NULL,
        scompany VARCHAR(20) NOT NULL,
        spid INT NOT NULL,
        sunit INT NOT NULL,
        FOREIGN KEY(spid) REFERENCES products(pid)
    ) AUTO_INCREMENT=101;
'''

In [24]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'

In [23]:
cur = conn.cursor()
cur.execute(sql_ct_sale)

0

In [3]:
calendar = {1:31,2:29,3:31,4:30,5:31,6:30,7:31,8:31,9:30,10:31,11:30,12:31}
company_list = ['멀티캠퍼스','한경아카데미','표준협회','생산성본부','테크브루']
for i in range(10):
    date = f'2020-1-{random.randint(1,31)}'
    company = company_list[random.randint(0,4)]
    pid = random.randint(11,40)
    unit = random.randint(1,20)
    print(date,company,pid,unit)

2020-1-27 생산성본부 38 18
2020-1-9 한경아카데미 39 2
2020-1-15 생산성본부 29 3
2020-1-11 표준협회 20 16
2020-1-3 한경아카데미 16 2
2020-1-24 멀티캠퍼스 32 18
2020-1-10 멀티캠퍼스 28 15
2020-1-4 한경아카데미 30 15
2020-1-21 표준협회 27 11
2020-1-22 표준협회 36 13


In [12]:
sql = "insert into sales values(default,%s,%s,%s,%s);"
for month, day in calendar.items():
    for i in range(10):
        date = f'2020-{month}-{random.randint(1,day)}'
        company = company_list[random.randint(0,4)]
        pid = random.randint(11,40)
        unit = random.randint(1,20)
        cur.execute(sql, (date, company, pid, unit))
conn.commit()

## 두 테이블을 Join 한 View 생성


In [13]:

sql = """
    CREATE VIEW sales_book as
        SELECT sid, sdate, scompany, pid, pname, pcategory, pprice, sunit, pcost,
            pprice*sunit AS revenue, (pprice-pcost)*sunit AS profit
        FROM sales
        JOIN products
        ON sales.spid = products.pid
        ORDER BY sdate;
"""


In [14]:
cur = conn.cursor()
cur.execute(sql)

0

- 데이터를 sales_book.csv로 저장


In [15]:
sql = 'select * from sales_book'
cur.execute(sql)
results = cur.fetchall()

In [16]:
import pandas as pd

df = pd.DataFrame(results)
df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,104,2020-01-03,한경아카데미,16,랜턴,캠핑용품,94600,2,89870,189200,9460
1,107,2020-01-04,한경아카데미,30,윤활방청제,자동차용품,57400,15,52810,861000,68850
2,101,2020-01-09,한경아카데미,39,로잉머신,헬스용품,69500,2,66020,139000,6960
3,106,2020-01-10,멀티캠퍼스,28,헤드유닛,자동차용품,21800,15,20710,327000,16350
4,103,2020-01-11,표준협회,20,워터저그,캠핑용품,75300,16,73790,1204800,24160


In [17]:
del df[3]
del df[0]



In [18]:
df.columns = ['판매일자','판매처','상품명','카테고리','단가','수량','원가','매출액','이익']
df.head()

,판매일자,판매처,상품명,카테고리,단가,수량,원가,매출액,이익
0,2020-01-03,한경아카데미,랜턴,캠핑용품,94600,2,89870,189200,9460
1,2020-01-04,한경아카데미,윤활방청제,자동차용품,57400,15,52810,861000,68850
2,2020-01-09,한경아카데미,로잉머신,헬스용품,69500,2,66020,139000,6960
3,2020-01-10,멀티캠퍼스,헤드유닛,자동차용품,21800,15,20710,327000,16350
4,2020-01-11,표준협회,워터저그,캠핑용품,75300,16,73790,1204800,24160


In [19]:
df.to_csv('sales_book.csv', index=False)

In [20]:
cur.close()
conn.close()